In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tokenizer import Tokenizer
from llama2_model import LLaMA

from safetensors import safe_open
from jax.sharding import Mesh, NamedSharding, PartitionSpec
import jax.numpy as jnp
import transformers
import numpy as np
import jax
import jmp

/home/neverix/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = Tokenizer("models/Llama-2-7b-hf/tokenizer.model")
input_ids = tokenizer.encode("Hello world!", bos=True, eos=False)
input_ids

[1, 15043, 3186, 29991]

In [4]:
reference_llama = transformers.LlamaModel.from_pretrained("models/Llama-2-7b-hf")

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.27it/s]


In [5]:
num_devices = len(jax.devices())
mesh = Mesh(np.array(jax.devices()).reshape(2, 4), axis_names=("dp", "mp"))
policy = jmp.get_policy("p=bf16,c=bf16")
llama = LLaMA(jax.random.PRNGKey(32), mesh, policy)

In [6]:
import equinox as eqx
import re


with safe_open(
    "/home/neverix/micrlhf/models/Llama-2-7b-hf/model-00001-of-00002.safetensors",
    framework="numpy",
    device="cpu",
) as f:
    for k in f.keys():
        weight = f.get_tensor(k)
        re_sult = re.search(r"layers\.([0-9]+)", k)
        try:
            k = (
                k[: re_sult.span()[0]]
                + f"layers[{re_sult.group(1)}]"
                + k[re_sult.span()[1] :]
            )
        except AttributeError:
            pass
        print("Loading", k)
        llama = eval(
            f"eqx.tree_at(lambda l: l.{k}, llama, jax.device_put(weight, src=llama.{k}))"
        )

Loading model.embed_tokens.weight
Loading model.layers[0].input_layernorm.weight
Loading model.layers[0].mlp.down_proj.weight
Loading model.layers[0].mlp.gate_proj.weight
Loading model.layers[0].mlp.up_proj.weight
Loading model.layers[0].post_attention_layernorm.weight
Loading model.layers[0].self_attn.k_proj.weight
Loading model.layers[0].self_attn.o_proj.weight
Loading model.layers[0].self_attn.q_proj.weight
Loading model.layers[0].self_attn.rotary_emb.inv_freq
Loading model.layers[0].self_attn.v_proj.weight
Loading model.layers[1].input_layernorm.weight
Loading model.layers[1].mlp.down_proj.weight
Loading model.layers[1].mlp.gate_proj.weight
Loading model.layers[1].mlp.up_proj.weight
Loading model.layers[1].post_attention_layernorm.weight
Loading model.layers[1].self_attn.k_proj.weight
Loading model.layers[1].self_attn.o_proj.weight
Loading model.layers[1].self_attn.q_proj.weight
Loading model.layers[1].self_attn.rotary_emb.inv_freq
Loading model.layers[1].self_attn.v_proj.weight
Lo

In [7]:
with mesh:
    input_ids = [
        tokenizer.encode(x, bos=True, eos=False)
        for x in ["Hello world", "This is a test"]
    ]
    input_ids = [x + [0] * (128 - len(x)) for x in input_ids]
    ids = jnp.asarray(input_ids)
    ids = jax.device_put(ids, NamedSharding(mesh, spec=PartitionSpec("dp", None)))
    print(llama(ids))

TypeError: reshape total size must be unchanged, got new_sizes (1, 128, 16, 1, 1) for shape (128, 64).